In [1]:
from ngsolve.meshes import *
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.geom2d import SplineGeometry
from netgen.geom2d import CSG2d, Circle, Rectangle
import matplotlib.pyplot as plt
import numpy as np
import time as timeit
from xfem import InterpolateToP1
from aux import *

ngsglobals.msg_level=1

def FVSolveBot(case=2, nx=50, order = 0, cfl=0.5, method ="RK2", 
               eps0 = 1e-16, epsRho=1e-3, maxV=0, 
               plot=False, tol = 0.05, flag="char",
              pcBot=False, limitB=True):
    time = 0
    count = 0
    if order <2:
        sd = 0
    elif order ==2:
        sd = 1
    
    # point evaluation matrix
    if order ==0:
        mat = [] # no eval pts
    elif order ==1: # three vertices
        mat = np.array([[-1,-1,2], 
                        [-1,1,0]])
    elif order==2: # three vertices + three midpts
        mat = np.array([[-1.,   -1.,    2.,    0.5 , -1. ,   0.5 ],
                        [ 1.,    1.,    3.,   -0.5 ,  1. ,  -0.5 ],
                        [-1.,    1.,    0.,   -0.5 ,  0. ,   0.5 ],
                        [ 1.,   -1.,    0.,   -0.75,  0. ,   0.75],
                        [ 1.,    1.,    0.,    0.25, -0.5,   0.25]]) 
    else:
        print("OPPS... eval mat not implemented yet")
        stop
    
    g = 9.812
    if case=='1a': # Ex5.8 wb
        tend = 0.1
        uI = 0*x
        vI = 0*x
        b0 = 0.8*exp(-50*((x-0.5)**2+(y-0.5)**2))
        rhobI = 1.0+0*x
        rhoI = rhobI-b0
        tgrid = [tend,20]
        h0 = 1/nx
        geo = SplineGeometry()
        geo.AddRectangle([0,0],[1,1])
        mesh = Mesh(geo.GenerateMesh(maxh=h0))
        
        outbc=""
        wallbc = ""
    elif case=='1b': # Ex5.9 small perturbation (half domain)
        tend = 0.60
        uI = 0*x
        vI = 0*x
        b0 = 0.8*exp(-5*(x-0.9)**2-50*(y-0.5)**2)
        delta = 1e-2
        rhobI = 1 + IfPos((x-0.05)*(x-0.15), 0, delta)
        rhoI = rhobI-b0
        tgrid = [0.12,0.24,0.36,0.48,0.60,20]
        h0 = 1/nx # mesh size: decrease it to 0.00625 later (THIS IS ALREADY slow)
        geo = SplineGeometry()
        geo.AddRectangle([0,0], [0.05,0.5])
        geo.AddRectangle([0.05,0], [0.15,0.5])
        geo.AddRectangle([0.15,0], [2,0.5])
        mesh = Mesh(geo.GenerateMesh(maxh=h0))
        mesh.ngmesh.SetBCName(9,"right")
        mesh.ngmesh.SetBCName(3,"left")
        
        outbc="left|right"
        wallbc = ""
    elif case=='1bx': # Ex5.9 small perturbation (full domain)
        tend = 0.60
        uI = 0*x
        vI = 0*x
        b0 = 0.8*exp(-5*(x-0.9)**2-50*(y-0.5)**2)
        delta = 1e-2
        rhobI = 1 + IfPos((x-0.05)*(x-0.15), 0, delta)
        rhoI = rhobI-b0
        tgrid = [0.12,0.24,0.36,0.48,0.60,20]
        h0 = 1/nx # mesh size: decrease it to 0.00625 later (THIS IS ALREADY slow)
        geo = SplineGeometry()
        geo.AddRectangle([0,0], [0.05,1])
        geo.AddRectangle([0.05,0], [0.15,1])
        geo.AddRectangle([0.15,0], [2,1])
        mesh = Mesh(geo.GenerateMesh(maxh=h0))
        mesh.ngmesh.SetBCName(9,"right")
        mesh.ngmesh.SetBCName(3,"left")
        
        outbc="left|right"
        wallbc = ""
    elif case=='1c': # circular dam: wet bed
        tend = 0.69
        
        uI = 0*x
        vI = 0*x
        b0 = 0*x
        rhobI = CF([1,10])
        rhoI = rhobI-b0
        tgrid = [tend,20]
        h0 = 25/nx # roughly 50 x 50 mesh
        geo = CSG2d()
        # define some primitives
        rect = Rectangle( pmin=(0,0), pmax=(25,25), mat="mat1", bc="wall" )
        circle = Circle( center=(0,0), radius=11, mat="mat1", bc="bc_circle")
        geo.Add(rect-circle)
        geo.Add(rect*circle)
        
        mesh = Mesh(geo.GenerateMesh(maxh=h0))
        outbc=""
        wallbc = "wall"
    elif case=='1cx': # circular dam: dry bed
        tend = 0.69
        
        uI = 0*x
        vI = 0*x
        b0 = 0*x
        rhobI = CF([eps0,10])
        rhoI = rhobI-b0
        tgrid = [tend,20]
        h0 = 25/nx # roughly 50 x 50 mesh
        geo = CSG2d()
        # define some primitives
        rect = Rectangle( pmin=(0,0), pmax=(25,25), mat="mat1", bc="wall" )
        circle = Circle( center=(0,0), radius=11, mat="mat1", bc="bc_circle")
        geo.Add(rect-circle)
        geo.Add(rect*circle)
        
        mesh = Mesh(geo.GenerateMesh(maxh=h0))
        outbc=""
        wallbc = "wall"
    elif case=='1d': # Ex4.7 three mounds: wall bc (only simulate half domain)
        tend = 40
        m1 = 1-0.1*((x-30)**2+(y-22.5)**2)**0.5
        m2 = 1-0.1*((x-30)**2+(y-7.5)**2)**0.5
        m3 = 2.8-0.28*((x-47.5)**2+(y-15)**2)**0.5
        mA = IfPos(m1, m1, 0)
        mB = IfPos(mA-m2, mA, m2)
        # the bottom
        b0 = IfPos(mB-m3, mB, m3)
        tgrid = [5,10,15,20,25,30,35,tend,600]
        h0 = 30/nx # roughly 70 x 30 mesh
        geo = CSG2d()
        # define some primitives
        rect1 = Rectangle( pmin=(0,15), pmax=(16,30), mat="mat1", bc="wall" )
        rect2 = Rectangle( pmin=(16,15), pmax=(70,30), mat="mat2", bc="wall" )
        geo.Add(rect1)
        geo.Add(rect2)

        mesh = Mesh(geo.GenerateMesh(maxh=h0))
        mesh.ngmesh.SetBCName(1,"int")
        mesh.ngmesh.SetBCName(7,"int")
        rhoI = CF([1.875,eps0])

        uI = 0*x
        vI = 0*x
        rhobI = rhoI+b0
        outbc=""
        wallbc = "wall"            
            
    fileX = "case"+case+"P"+str(order)+"N"+str(nx)
    eps = (2/g)**0.5
    nelems = mesh.ne    
    
    n = specialcf.normal(mesh.dim)
    # Burgers
    V = L2(mesh, order=order, all_dofs_together=False)
    fes = FESpace([V,V,V])

    # TVB limiter preparation
    listNbr, listNbj, listNbw, alphaList, listNrms = getNeighbors2D(mesh)

    bh = GridFunction(V)
    if pcBot==True:
        # use L2 projection (PC)
        tmpB = GridFunction(L2(mesh))
        rho0 = GridFunction(L2(mesh))
    else:
        # continuous Bottom
        tmpB = GridFunction(H1(mesh,order=order))
        rho0 = GridFunction(H1(mesh,order=order))
            
    gfu = GridFunction(fes)
    gfu0 = GridFunction(fes)
    
    rhoh, uh1, uh2 = gfu.components
    rhoh0, uh10, uh20 = gfu0.components
    rhoh1 = GridFunction(V)
    mh1 = GridFunction(V) # momentum
    mh2 = GridFunction(V) # momentum
    mh10 = GridFunction(V) # momentum
    mh20 = GridFunction(V) # momentum
    
    if case=="1d" and pcBot==False: # nodal interpolation
        tmpB =  GridFunction(H1(mesh))
        InterpolateToP1(b0, tmpB)
        bh.Set(tmpB)
        rhoh.Set(rhoI) 
    else:
        tmpB.Set(b0)
        bh.Set(tmpB)
        rho0.Set(rhobI)
        rhoh.Set(rho0) # THIS INTRODUCE NUMERICAL ERROR !    
        rhoh.vec.data -= bh.vec # remove heights

        
    uh1.Set(uI)
    uh2.Set(vI)
            
    # add mh
    mh1.vec.data = uh1.vec
    V.ApplyM(rho=rhoh, vec=mh1.vec)
    V.SolveM(rho=1, vec=mh1.vec)
    mh2.vec.data = uh2.vec
    V.ApplyM(rho=rhoh, vec=mh2.vec)
    V.SolveM(rho=1, vec=mh2.vec)
    
    # rescale
    epsRho *= max(rhoh.vec[:nelems].FV().NumPy())
    print("Low-order cut:", epsRho)
        
    (rho, u1, u2), (eta, v1, v2) = fes.TnT()
    rhoO, u1O, u2O = rho.Other(rhoI), u1.Other(uI), u2.Other(vI)
    etaO, v1O, v2O = eta.Other(), v1.Other(), v2.Other()
    
    u = CF((u1,u2))
    v = CF((v1,v2))
    uO = CF((u1O,u2O))
    vO = CF((v1O,v2O))
    du = CF((grad(u1), grad(u2)), dims=(2,2))
    dv = CF((grad(v1), grad(v2)), dims=(2,2))
    
    # Hydro static reconstruction
    if order==0 or pcBot==True: # THIS GUARANTEE POSITIVITY
        maxB = IfPos(bh-bh.Other(), bh, bh.Other())
        rhoL = IfPos(rho+bh-maxB, rho+bh-maxB, 0)
        rhoR = IfPos(rhoO+bh.Other()-maxB, rhoO+bh.Other()-maxB, 0)
        jmp_rho = rhoL-rhoR
        avg_rho = 0.5*(rhoL+rhoR)
        cpFlag=False
    else: # continuous bottom
        rhoL = rho
        rhoR = rhoO
        jmp_rho = rho-rhoO #+bh-bh.Other()
        avg_rho = 0.5*(rhoL+rhoR)+bh
        cpFlag=False # NOTE: you may need to set this to False
    
    cL = (2*rhoL)**0.5/eps
    cR = (2*rhoR)**0.5/eps

    avg_rho = 0.5*(rho+rhoO)
    avg_u = 0.5*(u+uO)
    
    # Later!!!
    unL = u*n
    unR = uO*n
    abs_uL = IfPos(unL, unL, -unL)
    abs_uR = IfPos(unR, unR, -unR)
    
    speedL = cL+abs_uL
    speedR = cR+abs_uR

    ### ESTIMATE SPEED ?! FIXME LATER
    speed_rho = IfPos(speedL-speedR, speedL, speedR)    
    
    # EEC flux + stab
    flux_rho = 0.5*(rhoL*u+rhoR*uO)*n+0.5*speed_rho*jmp_rho
    # FIXME LATER
    flux_m = flux_rho*avg_u + 0.5*speed_rho*avg_rho*(u-uO)
        
    # Spatial operator (for rho updates)
    a = BilinearForm(fes, nonassemble=True)
    # volume contributions (skew symmetric)
    if order>0:
        a += ((-rho*u*grad(eta)
              +g*(grad(rho)+grad(bh))*rho*v
              -0.5*rho*dv*u*u+0.5*rho*du*u*v
              )*dx).Compile(cpFlag, True)
    # interior bdry contribution
    a += ((flux_rho*(eta-etaO)
          +flux_m*(v-vO)
           -0.5*flux_rho*(u*v-uO*vO)
          -g*jmp_rho*n*0.5*(rhoL*v+rhoR*vO)
          )*dx(skeleton=True)).Compile(cpFlag, True)
    # Boundary parts (FIXME)
    a += ((flux_rho*eta
          +flux_m*v
          -0.5*flux_rho*u*v
          -g*jmp_rho*n*rho*v
         )*ds(skeleton=True, definedon=mesh.Boundaries(outbc))).Compile(cpFlag, True)
    
    # wall bdry stabilization
    a += (speedL*(rhoL+bh)*u*n*v*n*ds(skeleton=True, definedon=mesh.Boundaries(wallbc))).Compile(cpFlag, True)
    
    tmp = gfu.vec.CreateVector()
   
    tmpU = uh1.vec.CreateVector()
    nrho = V.ndof

    step = 0
    
    # velocity estimation
    vel = GridFunction(L2(mesh))
    vel.Set((2*rhoh)**0.5/eps+(uh1*uh1+uh2*uh2)**0.5)        
    dt = min(cfl*h0/max(vel.vec), 0.1)
    
    def ForwardEuler(dt):
        # update height
        a.Apply(gfu.vec, tmp)
        # update derivative OLD part
        tmpU.data = uh1.vec
        V.ApplyM(rho=rhoh, vec=tmpU)
        tmp[nrho:2*nrho].data += 0.5/dt*tmpU
        tmpU.data = uh2.vec
        V.ApplyM(rho=rhoh, vec=tmpU)
        tmp[2*nrho:].data += 0.5/dt*tmpU
        
        # update density (unlimited)        
        V.SolveM(rho=1, vec=tmp[:nrho])
        rhoh.vec.data -= dt*tmp[:nrho]

        # update derivative NEW part
        tmpU.data = uh1.vec
        V.ApplyM(rho=rhoh, vec=tmpU)
        tmp[nrho:2*nrho].data -= 0.5/dt*tmpU

        tmpU.data = uh2.vec
        V.ApplyM(rho=rhoh, vec=tmpU)
        tmp[2*nrho:].data -= 0.5/dt*tmpU

        # update momentum (unlimited)
        V.SolveM(rho=1, vec=tmp[nrho:2*nrho])
        mh1.vec.data -= dt*tmp[nrho:2*nrho]

        # update momentum (unlimited)
        V.SolveM(rho=1, vec=tmp[2*nrho:])
        mh2.vec.data -= dt*tmp[2*nrho:]
            
    # convex combination
    def ConvexComb(w0, w1):
        # update density
        rhoh.vec.data *= w1
        rhoh.vec.data += w0*rhoh0.vec

        # update momentum
        mh1.vec.data *= w1
        mh1.vec.data += w0*mh10.vec

        # update momentum
        mh2.vec.data *= w1
        mh2.vec.data += w0*mh20.vec

    def CalcVel():
        tmpU.data = mh1.vec
        V.ApplyM(rho=1, vec=tmpU)
        V.SolveM(rho=rhoh, vec=tmpU)
        uh1.vec.data = tmpU
        
        tmpU.data = mh2.vec
        V.ApplyM(rho=1, vec=tmpU)
        V.SolveM(rho=rhoh, vec=tmpU)
        uh2.vec.data = tmpU
        if maxV>0:
            nlimit = limitU2D(uh1, uh2, nelems, order, mat, listNbr, maxV)
        else:
            nlimit = 0
        return nlimit
    
    # limit den + mh
    def Limit():
        if order >0 and epsRho > 0:
            # (1) avg_rho < epsRho: use lowest order P0 approximation
            drylimit2D(rhoh, mh1, mh2, order, nelems, epsRho=epsRho)
        
        # (2) tvb limit
        maskM, nlimit = tvbLimitM2D(rhoh, bh, mh1, mh2, order, mesh, 
                                    listNbr, listNbj, listNbw, alphaList, listNrms,
                                    nu=1.2, tol = tol, flag=flag, g=g, epsRho=epsRho, 
                                    limitB=limitB)
            
        # (3) pplimit
        pplimit(rhoh, order, mips, npts,  nelems, mw, wt, mesh.dim, eps0=eps0)
        return maskM, nlimit
            
    
    # PP limiter preparation
    N = ceil((3+order)/2)
    mw = 1/N/(N-1) # lobato quad weights
    ir = IntegrationRule(SEGM, 2*order+1)
    wt = ir.weights
    npts = len(wt)*3
    mips = getVerts2D(mesh, order)
    
    vtk = VTKOutput(ma=mesh,coefs=[rhoh+bh, uh1, uh2],  names=["height", "ux", "uy"],
                    filename=fileX, subdivision=sd)
    
    if plot==True:
#         gfi = GridFunction(L2(mesh))
#         gfi0 = gfi.vec.FV().NumPy()
#         scene = Draw(gfi, mesh,"soln",deformation=True) 
#         scene = Draw(10*rhoh, mesh,"soln",deformation=True) 
#         scene = Draw(10*(rhoh+bh), mesh,"soln",deformation=True) 
        scene = Draw(uh1, mesh,"soln", deformation=True, max=maxV, autoscale=False) 
    mask, nlimit = 0,0
    reject = 0
    with TaskManager():
        while time < tend-dt/2:
            if dt < 1e-5:
                print(reject, dt, "TOO SMALL dt")
                break                
            time += dt
            if time > tend:
                time -=dt
                dt = tend-time
                time = tend
            step += 1
            gfu0.vec.data = gfu.vec
            mh10.vec.data = mh1.vec
            mh20.vec.data = mh2.vec

            ### RK3 stage 1
            ForwardEuler(dt)
            if order > 0: # limit first set always
                mask, nlimit = Limit()
            # update vel
            nlimitU = CalcVel()
            # check min rho
            rhomin = min(rhoh.vec[:nelems])
            if rhomin < 0 :
#                 print("Stage 1 @step: %4.0f time: %2.3e, rhoMin:%.2e, dt:%.2e"%(step, time, rhomin, dt))
                # Half time step
                gfu.vec.data = gfu0.vec
                mh1.vec.data = mh10.vec
                mh2.vec.data = mh20.vec
                time -= dt
                step -= 1
                dt /= 2
                reject += 1
                continue
            
            ## RK2
            if method !="RK1":
                ForwardEuler(dt)                
                rhomin = min(rhoh.vec[:nelems])
                if rhomin < 0 :
#                     print("Stage 2 @step: %4.0f time: %2.3e, rhoMin:%.2e, dt:%.2e "%(step, time, rhomin,dt))
                    # Half time step
                    gfu.vec.data = gfu0.vec
                    mh1.vec.data = mh10.vec
                    mh2.vec.data = mh20.vec
                    time -= dt
                    step -= 1
                    dt /= 2    
                    reject += 1
                    continue

                # RK2 update: HOW TO KEEP momentum conservation?
                if method=="RK2":
                    # USE PROJECTIONS
                    ConvexComb(0.5, 0.5)
                    Limit()
                    nlimitU = CalcVel()
                else: 
                    # USE PROJECTIONS
                    ConvexComb(0.75, 0.25)
                    Limit()
                    nlimitU = CalcVel()
                
                    # Stage 3
                    ForwardEuler(dt)
                    rhomin = min(rhoh.vec[:nelems])
                    if rhomin<0 :
#                         print("Stage 3 @step: %4.0f time: %2.3e, rhoMin:%.2e, dt:%.2e "%(step, time, 
#                                                                                                rhomin,dt))
                        # Half time step
                        gfu.vec.data = gfu0.vec
                        mh1.vec.data = mh10.vec
                        mh2.vec.data = mh20.vec
                        time -= dt
                        step -= 1
                        dt /= 2
                        reject += 1
                        continue
                    # convex combination
                    # USE PROJECTIONS
                    ConvexComb(1/3, 2/3)
                    mask, nlimit = Limit()
                    nlimitU = CalcVel()
                    
            # estimate based on cell-averages        
            vel.Set((2*rhoh)**0.5/eps+(uh1**2+uh2**2)**0.5)
            vmax = max(vel.vec)
            dt = cfl*h0/vmax
            rho0 = min(rhoh.vec[:mesh.ne])
            print("\r", format(time,'.4e'), "STEP: ", format(step,'4.0f'), 
                  "Nlimit: %4.f %4.f"%(nlimit, nlimitU),
                  "dt: ", format(dt,'.3e'), 
                  "rhomin:", format(rho0, '.3e'),"vmax:", format(vmax, '.3e'), 
                  "rej:", format(reject, "3.0f"), end="")
            if time > tgrid[count]-dt/2:
                count += 1
                vtk.Do()
                if plot==True:
                    scene.Redraw() 
            if plot==True and step%10==0:
                # compute indicator
#                 gfi.vec.FV().NumPy()[:]=mask
#                 gfi.vec.data=uh1.vec[:nelems]
                scene.Redraw()       
        return rhoh, bh, uh1, uh2, mh1, mh2

In [2]:
case = '1a'
plot = False
k0 = 2

nx = 50
tol = 0.02 # THIS TVB indicator tolerance
flag = "char" # THIS IS TVB limiting flag

eps0 = 1e-12 # minimal height

# THIS HACK IS NEEDED FOR 4a only
maxV = 0
epsRho = 0 # minimal percentage of height
if case == "1cx":
    epsRho = 5e-3 # NOTE: If rho< epsRho, go to P0 approx
    maxV = 15
    
if case=="1d":
    epsRho = 1e-3 # NOTE: If rho< epsRho, go to P0 approx
    maxV = 9


pcBot = False
limitB = True # limit (h+b, m) as indicator 

for k in range(k0, k0+1):
    print("\n Start P",k)
    if k==0:
        method = "RK1"
        cfl0 = 0.2
    elif k==1:
        method = "RK3"
        cfl0 = 0.1
    elif k==2:
        method = "RK3"
        cfl0 = 0.05 # must be < 0.1
        
    rhoh, bh, uh1, uh2, mh1, mh2 = FVSolveBot(case=case, nx = nx, order=k, cfl = cfl0, method=method, 
                                          eps0=eps0, epsRho=epsRho, maxV=maxV, plot=plot, 
                                          tol=tol, flag=flag, pcBot=pcBot, limitB=limitB)


 Start P 2
Low-order cut: 0.0
 6.0000e-01 STEP:  3786 Nlimit:   53    0 dt:  1.587e-04 rhomin: 2.006e-01 vmax: 3.150e+00 rej:   0

In [3]:
Draw(rhoh, rhoh.space.mesh)

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


WebGuiWidget(value={'ngsolve_version': '6.2.2105-56-gbd51b27', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'dra…

BaseWebGuiScene

In [4]:
np.linspace(0.01,8.9,12)

array([0.01      , 0.81818182, 1.62636364, 2.43454545, 3.24272727,
       4.05090909, 4.85909091, 5.66727273, 6.47545455, 7.28363636,
       8.09181818, 8.9       ])

In [5]:
np.linspace(2,9.4,11)

array([2.  , 2.74, 3.48, 4.22, 4.96, 5.7 , 6.44, 7.18, 7.92, 8.66, 9.4 ])

In [6]:
9.4-0.74*10

2.0